In [1]:
import pandas as pd #data processing for csv files
import geopandas as gpd #data processing for shapefiles 

In [14]:
#download and read all the files
df_prjct_schl = pd.read_csv('https://data.cityofnewyork.us/api/views/a94k-kjys/rows.csv?accessType=DOWNLOAD')
df_prjct_loc = pd.read_csv('https://data.cityofnewyork.us/api/views/tesz-9suw/rows.csv?accessType=DOWNLOAD')
df_schl_brgh = pd.read_csv('https://data.cityofnewyork.us/api/views/esmb-8zkm/rows.csv?accessType=DOWNLOAD')
shp_schl = gpd.read_file("https://data.cityofnewyork.us/api/views/jfju-ynrr/files/IjLvNcAZDM01muXoNu_-a2ggw13oNeeV4xH-rPS_Dig?filename=Public_School_Locations.zip")

In [5]:
#display the column names of 'Capacity projects by school'
df_prjct_schl.columns

Index(['Sited', 'Leased', 'District', 'Project #', 'School', 'Boro',
       'Forecast Capacity', 'Design Start', 'Construction Start',
       'Actual Est. Comp', 'Total Est. Comp', 'Previous Appropriations',
       'Funding Req'D FY 20-24', 'Needed to Complete'],
      dtype='object')

In [7]:
#display the column names of 'Capacity project site location'
df_schl_brgh.columns

Index(['District', 'Borough', 'Building ID', 'School', 'Project #',
       'Description', 'FY', 'Total'],
      dtype='object')

In [8]:
#Create a joined table for capacity projects
df_capacity = pd.merge(df_prjct_schl, df_prjct_loc, how='left', left_on='School', right_on='SCHOOL')

In [9]:
#drop duplicated columns and the empty column'Boro'
df_capacity = df_capacity.drop(columns = ['Boro', 'SCHOOL','DISTRICT'])

#rename the column names to a consistent format
df_capacity.rename(columns={'BOROUGH':'Borough','LOCATION':'Location'}, inplace=True)

In [10]:
#merge the tables of capacity projects and capital projects
df_prjct = pd.concat([df_schl_brgh,df_capacity], axis=0, join='outer',sort=False)

In [11]:
#adding the point geometry to each project
shp_prjct = pd.merge(shp_schl[['SCHOOLNAME','ADDRESS','geometry']], 
                     df_prjct, how='right',left_on = 'SCHOOLNAME', right_on = 'School')

In [12]:
#drop duplicated columns and rename column names to a consistent format
shp_prjct = shp_prjct.drop(columns = ['SCHOOLNAME']).rename(columns={'ADDRESS':'Address'})

In [13]:
#display the final joined table
shp_prjct.head()

,Address,geometry,District,Borough,Building ID,School,Project #,Description,FY,Total,...,Location,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,601 OCEANVIEW AVENUE,POINT (994847.1202502615 150547.2376673446),21,Brooklyn,K253,P.S. 253,DSF0000912069,State of Good Repair - System Replacements - E...,2020.0,300000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1307 CENTRAL AVENUE,POINT (1053949.276883927 160637.15219117),21,Brooklyn,K253,P.S. 253,DSF0000912069,State of Good Repair - System Replacements - E...,2020.0,300000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,360 KEAP STREET,POINT (996991.9747995249 197833.0000531077),14,Brooklyn,K319,P.S. 319,DSF0000895802,State of Good Repair - System Replacements - R...,2020.0,1357439.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,360 KEAP STREET,POINT (996991.9747995249 197833.0000531077),14,Brooklyn,K319,P.S. 319,DSF0000895803,State of Good Repair - System Replacements - W...,2020.0,1314643.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,334 GREENWICH STREET,POINT (981231.783227806 200964.4655709098),30,Queens,Q150,P.S. 150,DSF0000902816,State of Good Repair - System Replacements - R...,2020.0,2389992.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
shp_prjct.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 392 entries, 0 to 391
Data columns (total 30 columns):
Address                    30 non-null object
geometry                   30 non-null object
District                   392 non-null object
Borough                    316 non-null object
Building ID                304 non-null object
School                     392 non-null object
Project #                  392 non-null object
Description                304 non-null object
FY                         304 non-null float64
Total                      304 non-null float64
Sited                      16 non-null object
Leased                     22 non-null object
Forecast Capacity          88 non-null float64
Design Start               88 non-null object
Construction Start         88 non-null object
Actual Est. Comp           88 non-null object
Total Est. Comp            88 non-null float64
Previous Appropriations    88 non-null float64
Funding Req'D FY 20-24     88 non-null float64

In [17]:
#generate the output tables
df_capacity.to_csv('capacity_projects.csv')
df_schl_brgh.to_csv('capital_projects.csv')

In [18]:
shp_prjct.to_file('capacity_capital_projects_location.shp', driver='ESRI Shapefile')

AttributeError: 'float' object has no attribute '__geo_interface__'